In [1]:
import numpy as np
import pandas as pd
import csv
from datetime import datetime, timedelta
from dateutil import rrule
import random

In [2]:
# read data
order_data = pd.read_csv("final_salesdata.txt", sep="\t")

# convert date to datetime
order_data['date'] = pd.to_datetime(order_data.date)
order_data

C:\Users\Maurice Jansen\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,orderID,date,storeID,customer_ID,total_price,quantity
0,O10000000,2003-01-01,1,0,129.90,2
1,O10000002,2003-01-01,1,0,109.90,2
2,O10000004,2003-01-02,1,0,59.95,1
3,O10000005,2003-01-02,1,0,59.95,1
4,O10000006,2003-01-02,1,0,54.95,1
...,...,...,...,...,...,...
1922076,O13613510,2021-12-31,3,0,224.80,4
1922077,O13613520,2021-12-31,6,0,119.90,2
1922078,O13613522,2021-12-31,1,0,64.95,1
1922079,O13613483,2021-12-31,2,0,49.95,1


In [3]:
ship_data = order_data[order_data.storeID == 7]
ship_data = ship_data.drop(['storeID', 'customer_ID', 'total_price', 'quantity'], axis=1)
ship_data = ship_data.rename({'date': 'order_date'}, axis=1)  # new method
ship_data = ship_data.reset_index()
ship_data = ship_data.drop(['index'], axis=1)
ship_data

,orderID,order_date
0,O10244963,2011-01-01
1,O10244907,2011-01-01
2,O10244887,2011-01-01
3,O10244899,2011-01-01
4,O10244910,2011-01-01
...,...,...
596174,O13613082,2021-12-31
596175,O13612598,2021-12-31
596176,O13612679,2021-12-31
596177,O13614060,2021-12-31


In [4]:
# function to calculate the next weekday for shipping (1 day later)
def next_weekday_ship(date):
    
    # probability it will take x days longer because of Christmas
    month = date.month
    day = date.day
    day_extra_christmas = 0
    day_before_christmas = (day == 20 or day == 21 or day == 22 or day == 23 or day == 24)
    if (month) == 12 and day_before_christmas:
        probs = [0.54, 0.335, 0.11, 0.01, 0.005]
        day_extra_christmas = np.random.choice([0, 1, 2, 3, 4], p=[probs[0], probs[1], probs[2], probs[3], probs[4]])
    
    # probability it will take x days longer
    year = date.year
    year_11_15 = (year == 2011 or year == 2012 or year == 2013 or year == 2014 or year == 2015)
    year_16 = year == 2016
    year_17 = year == 2017

    if year_11_15:
        probs = [0.99, 0.005, 0.0025, 0.00125, 0.00125]
    elif year_16:
        probs = [0.98, 0.01, 0.005, 0.003, 0.002]
    elif year_17:
        probs = [0.965, 0.015, 0.015, 0.0025, 0.0025]
    else:
        probs = [0.96, 0.015, 0.01, 0.008, 0.007]

    day_extra = np.random.choice([0, 1, 2, 3, 4], p=[probs[0], probs[1], probs[2], probs[3], probs[4]])
    
    date = date + timedelta(days=int(day_extra+day_extra_christmas))

    # for friday
    if date.weekday() == 4:
        date = date + timedelta(days = 3)
        
    # for saturday
    elif date.weekday() == 5:
        date = date + timedelta(days = 3)

    # for sunday
    elif date.weekday() == 6:
        date = date + timedelta(days = 2)

    else: 
        date = date + timedelta(days = 1)
    
    return date

next_weekday_ship(datetime(2021,12,24))

datetime.datetime(2021, 12, 27, 0, 0)

In [5]:
# function to calculate the next weekday for arrival (2 days later)
def next_weekday_arrival(date):
    
    # probability it will take x days longer because of Christmas
    month = date.month
    day = date.day
    day_extra_christmas = 0
    day_before_christmas = (day == 20 or day == 21 or day == 22 or day == 23 or day == 24)
    if (month) == 12 and day_before_christmas:
        probs = [0.54, 0.335, 0.11, 0.01, 0.005]
        day_extra_christmas = np.random.choice([0, 1, 2, 3, 4], p=[probs[0], probs[1], probs[2], probs[3], probs[4]])
    
    # probability it will take x days longer
    year = date.year
    year_11_15 = (year == 2011 or year == 2012 or year == 2013 or year == 2014 or year == 2015)
    year_16 = year == 2016
    year_17 = year == 2017

    if year_11_15:
        probs = [0.99, 0.005, 0.0025, 0.00125, 0.00125]
    elif year_16:
        probs = [0.98, 0.005, 0.01, 0.0025, 0.0025]
    elif year_17:
        probs = [0.97, 0.015, 0.01, 0.003, 0.002]
    else:
        probs = [0.92, 0.06, 0.015, 0.0025, 0.0025]
    
    day_extra = np.random.choice([0, 1, 2, 3, 4], p=[probs[0], probs[1], probs[2], probs[3], probs[4]])
    date = date + timedelta(days=int(day_extra+day_extra_christmas))
    
    # for thursday
    if date.weekday() == 3:
        date = date + timedelta(days = 4)

    # for friday
    elif date.weekday() == 4:
        date = date + timedelta(days = 4)
        
    # for saturday
    elif date.weekday() == 5:
        date = date + timedelta(days = 4)

    # for sunday
    elif date.weekday() == 6:
        date = date + timedelta(days = 3)

    else: 
        date = date + timedelta(days = 2)

    return date

next_weekday_arrival(datetime(2021,12,23))

datetime.datetime(2021, 12, 29, 0, 0)

In [6]:
# add 'shipping date' to dataframe
ship_data['ship_date'] = ship_data.order_date.apply(lambda date: next_weekday_ship(date))
ship_data

,orderID,order_date,ship_date
0,O10244963,2011-01-01,2011-01-04
1,O10244907,2011-01-01,2011-01-04
2,O10244887,2011-01-01,2011-01-04
3,O10244899,2011-01-01,2011-01-04
4,O10244910,2011-01-01,2011-01-04
...,...,...,...
596174,O13613082,2021-12-31,2022-01-03
596175,O13612598,2021-12-31,2022-01-03
596176,O13612679,2021-12-31,2022-01-03
596177,O13614060,2021-12-31,2022-01-05


In [7]:
# add 'arrival date' to dataframe
ship_data['arrival_date'] = ship_data.ship_date.apply(lambda date: next_weekday_arrival(date))
ship_data

,orderID,order_date,ship_date,arrival_date
0,O10244963,2011-01-01,2011-01-04,2011-01-06
1,O10244907,2011-01-01,2011-01-04,2011-01-06
2,O10244887,2011-01-01,2011-01-04,2011-01-06
3,O10244899,2011-01-01,2011-01-04,2011-01-06
4,O10244910,2011-01-01,2011-01-04,2011-01-07
...,...,...,...,...
596174,O13613082,2021-12-31,2022-01-03,2022-01-06
596175,O13612598,2021-12-31,2022-01-03,2022-01-05
596176,O13612679,2021-12-31,2022-01-03,2022-01-05
596177,O13614060,2021-12-31,2022-01-05,2022-01-07


In [8]:
# function to calculate the difference in bussiness days
# (!) same way as we calculate the days of delivery, so not according to 'np.busday_count(start, end)'
def calc_dif_days(row, x, column1, column2):
    date1 = row[column1].date()
    date2 = row[column2].date()
    
    date1 = datetime(date1.year, date1.month, date1.day)
    date2 = datetime(date2.year, date2.month, date2.day)
    
    rule = rrule.rrule(dtstart=date1, freq=rrule.DAILY,
        byweekday=[rrule.MO, rrule.TU, rrule.WE, rrule.TH, rrule.FR])
    rule.between(date1, date2, inc=True)
    dif = len(rule.between(date1, date2, inc=True)) - 1  
    if dif > x:
        return dif, False
    else:
        return dif, True

In [9]:
# add 'total_delivery_days' column to dataframe
ship_data['total_delivery_days'] = ship_data.apply(lambda row: calc_dif_days(row, 3, 'order_date', 'arrival_date')[0], axis=1)
ship_data

,orderID,order_date,ship_date,arrival_date,total_delivery_days
0,O10244963,2011-01-01,2011-01-04,2011-01-06,3
1,O10244907,2011-01-01,2011-01-04,2011-01-06,3
2,O10244887,2011-01-01,2011-01-04,2011-01-06,3
3,O10244899,2011-01-01,2011-01-04,2011-01-06,3
4,O10244910,2011-01-01,2011-01-04,2011-01-07,4
...,...,...,...,...,...
596174,O13613082,2021-12-31,2022-01-03,2022-01-06,4
596175,O13612598,2021-12-31,2022-01-03,2022-01-05,3
596176,O13612679,2021-12-31,2022-01-03,2022-01-05,3
596177,O13614060,2021-12-31,2022-01-05,2022-01-07,5


In [10]:
# add 'total_delivery_days_too_late' column to dataframe
ship_data['total_delivery_days_too_late'] = ship_data.total_delivery_days.apply(lambda x: x - 3)
ship_data

,orderID,order_date,ship_date,arrival_date,total_delivery_days,total_delivery_days_too_late
0,O10244963,2011-01-01,2011-01-04,2011-01-06,3,0
1,O10244907,2011-01-01,2011-01-04,2011-01-06,3,0
2,O10244887,2011-01-01,2011-01-04,2011-01-06,3,0
3,O10244899,2011-01-01,2011-01-04,2011-01-06,3,0
4,O10244910,2011-01-01,2011-01-04,2011-01-07,4,1
...,...,...,...,...,...,...
596174,O13613082,2021-12-31,2022-01-03,2022-01-06,4,1
596175,O13612598,2021-12-31,2022-01-03,2022-01-05,3,0
596176,O13612679,2021-12-31,2022-01-03,2022-01-05,3,0
596177,O13614060,2021-12-31,2022-01-05,2022-01-07,5,2


In [11]:
# function to check whether original value is higher than value to compare with and return right value
def yes_or_no(ori_value, comp_value):
    if ori_value > comp_value:
        return False
    else:
        return True

In [12]:
# add 'delivery_on_time' column to dataframe
ship_data['total_delivery_on_time'] = ship_data.total_delivery_days.apply(lambda x: yes_or_no(x, 3))
ship_data

,orderID,order_date,ship_date,arrival_date,total_delivery_days,total_delivery_days_too_late,total_delivery_on_time
0,O10244963,2011-01-01,2011-01-04,2011-01-06,3,0,True
1,O10244907,2011-01-01,2011-01-04,2011-01-06,3,0,True
2,O10244887,2011-01-01,2011-01-04,2011-01-06,3,0,True
3,O10244899,2011-01-01,2011-01-04,2011-01-06,3,0,True
4,O10244910,2011-01-01,2011-01-04,2011-01-07,4,1,False
...,...,...,...,...,...,...,...
596174,O13613082,2021-12-31,2022-01-03,2022-01-06,4,1,False
596175,O13612598,2021-12-31,2022-01-03,2022-01-05,3,0,True
596176,O13612679,2021-12-31,2022-01-03,2022-01-05,3,0,True
596177,O13614060,2021-12-31,2022-01-05,2022-01-07,5,2,False


In [13]:
# add 'ship_days' column to dataframe
ship_data['ship_days'] = ship_data.apply(lambda row: calc_dif_days(row, 3, 'order_date', 'ship_date')[0], axis=1)
ship_data

,orderID,order_date,ship_date,arrival_date,total_delivery_days,total_delivery_days_too_late,total_delivery_on_time,ship_days
0,O10244963,2011-01-01,2011-01-04,2011-01-06,3,0,True,1
1,O10244907,2011-01-01,2011-01-04,2011-01-06,3,0,True,1
2,O10244887,2011-01-01,2011-01-04,2011-01-06,3,0,True,1
3,O10244899,2011-01-01,2011-01-04,2011-01-06,3,0,True,1
4,O10244910,2011-01-01,2011-01-04,2011-01-07,4,1,False,1
...,...,...,...,...,...,...,...,...
596174,O13613082,2021-12-31,2022-01-03,2022-01-06,4,1,False,1
596175,O13612598,2021-12-31,2022-01-03,2022-01-05,3,0,True,1
596176,O13612679,2021-12-31,2022-01-03,2022-01-05,3,0,True,1
596177,O13614060,2021-12-31,2022-01-05,2022-01-07,5,2,False,3


In [14]:
# add 'ship_days_too_late' column to dataframe
ship_data['ship_days_too_late'] = ship_data.ship_days.apply(lambda x: x - 1)
ship_data

,orderID,order_date,ship_date,arrival_date,total_delivery_days,total_delivery_days_too_late,total_delivery_on_time,ship_days,ship_days_too_late
0,O10244963,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0
1,O10244907,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0
2,O10244887,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0
3,O10244899,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0
4,O10244910,2011-01-01,2011-01-04,2011-01-07,4,1,False,1,0
...,...,...,...,...,...,...,...,...,...
596174,O13613082,2021-12-31,2022-01-03,2022-01-06,4,1,False,1,0
596175,O13612598,2021-12-31,2022-01-03,2022-01-05,3,0,True,1,0
596176,O13612679,2021-12-31,2022-01-03,2022-01-05,3,0,True,1,0
596177,O13614060,2021-12-31,2022-01-05,2022-01-07,5,2,False,3,2


In [15]:
# add 'ship_on_time' column to dataframe
ship_data['ship_on_time'] = ship_data.ship_days.apply(lambda x: yes_or_no(x, 1))
ship_data

,orderID,order_date,ship_date,arrival_date,total_delivery_days,total_delivery_days_too_late,total_delivery_on_time,ship_days,ship_days_too_late,ship_on_time
0,O10244963,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0,True
1,O10244907,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0,True
2,O10244887,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0,True
3,O10244899,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0,True
4,O10244910,2011-01-01,2011-01-04,2011-01-07,4,1,False,1,0,True
...,...,...,...,...,...,...,...,...,...,...
596174,O13613082,2021-12-31,2022-01-03,2022-01-06,4,1,False,1,0,True
596175,O13612598,2021-12-31,2022-01-03,2022-01-05,3,0,True,1,0,True
596176,O13612679,2021-12-31,2022-01-03,2022-01-05,3,0,True,1,0,True
596177,O13614060,2021-12-31,2022-01-05,2022-01-07,5,2,False,3,2,False


In [16]:
# add 'delivery_days' column to dataframe
ship_data['delivery_days'] = ship_data.apply(lambda row: calc_dif_days(row, 3, 'ship_date', 'arrival_date')[0], axis=1)
ship_data

,orderID,order_date,ship_date,arrival_date,total_delivery_days,total_delivery_days_too_late,total_delivery_on_time,ship_days,ship_days_too_late,ship_on_time,delivery_days
0,O10244963,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0,True,2
1,O10244907,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0,True,2
2,O10244887,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0,True,2
3,O10244899,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0,True,2
4,O10244910,2011-01-01,2011-01-04,2011-01-07,4,1,False,1,0,True,3
...,...,...,...,...,...,...,...,...,...,...,...
596174,O13613082,2021-12-31,2022-01-03,2022-01-06,4,1,False,1,0,True,3
596175,O13612598,2021-12-31,2022-01-03,2022-01-05,3,0,True,1,0,True,2
596176,O13612679,2021-12-31,2022-01-03,2022-01-05,3,0,True,1,0,True,2
596177,O13614060,2021-12-31,2022-01-05,2022-01-07,5,2,False,3,2,False,2


In [17]:
# add 'delivery_days_too_late' column to dataframe
ship_data['delivery_days_too_late'] = ship_data.delivery_days.apply(lambda x: x - 2)
ship_data

,orderID,order_date,ship_date,arrival_date,total_delivery_days,total_delivery_days_too_late,total_delivery_on_time,ship_days,ship_days_too_late,ship_on_time,delivery_days,delivery_days_too_late
0,O10244963,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0,True,2,0
1,O10244907,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0,True,2,0
2,O10244887,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0,True,2,0
3,O10244899,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0,True,2,0
4,O10244910,2011-01-01,2011-01-04,2011-01-07,4,1,False,1,0,True,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...
596174,O13613082,2021-12-31,2022-01-03,2022-01-06,4,1,False,1,0,True,3,1
596175,O13612598,2021-12-31,2022-01-03,2022-01-05,3,0,True,1,0,True,2,0
596176,O13612679,2021-12-31,2022-01-03,2022-01-05,3,0,True,1,0,True,2,0
596177,O13614060,2021-12-31,2022-01-05,2022-01-07,5,2,False,3,2,False,2,0


In [18]:
# add 'delivery_on_time' column to dataframe
ship_data['delivery_on_time'] = ship_data.delivery_days.apply(lambda x: yes_or_no(x, 2))
ship_data

,orderID,order_date,ship_date,arrival_date,total_delivery_days,total_delivery_days_too_late,total_delivery_on_time,ship_days,ship_days_too_late,ship_on_time,delivery_days,delivery_days_too_late,delivery_on_time
0,O10244963,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0,True,2,0,True
1,O10244907,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0,True,2,0,True
2,O10244887,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0,True,2,0,True
3,O10244899,2011-01-01,2011-01-04,2011-01-06,3,0,True,1,0,True,2,0,True
4,O10244910,2011-01-01,2011-01-04,2011-01-07,4,1,False,1,0,True,3,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
596174,O13613082,2021-12-31,2022-01-03,2022-01-06,4,1,False,1,0,True,3,1,False
596175,O13612598,2021-12-31,2022-01-03,2022-01-05,3,0,True,1,0,True,2,0,True
596176,O13612679,2021-12-31,2022-01-03,2022-01-05,3,0,True,1,0,True,2,0,True
596177,O13614060,2021-12-31,2022-01-05,2022-01-07,5,2,False,3,2,False,2,0,True


In [19]:
# distribution of total delivery days
print('Distribution of days of total delivery:')
print(ship_data['total_delivery_days'].value_counts())
print('')
print('Distribution of total deliveries on time:')
print(ship_data['total_delivery_on_time'].value_counts())

Distribution of days of total delivery:
3     537229
4      38556
5      14168
6       4163
7       1893
8        134
9         25
10         7
11         3
12         1
Name: total_delivery_days, dtype: int64

Distribution of total deliveries on time:
True     537229
False     58950
Name: total_delivery_on_time, dtype: int64


In [20]:
# distribution of shipping days
print('Distribution of days of shipment:')
print(ship_data['ship_days'].value_counts())
print('')
print('Distribution of shipment on time:')
print(ship_data['ship_on_time'].value_counts())

Distribution of days of shipment:
1    575445
2     11318
3      6130
4      2388
5       881
6        17
Name: ship_days, dtype: int64

Distribution of shipment on time:
True     575445
False     20734
Name: ship_on_time, dtype: int64


In [21]:
# distribution of delivery days
print('Distribution of days of delivery:')
print(ship_data['delivery_days'].value_counts())
print('')
print('Distribution of deliveries on time:')
print(ship_data['delivery_on_time'].value_counts())

Distribution of days of delivery:
2    555652
3     30493
4      7901
5      1326
6       804
7         3
Name: delivery_days, dtype: int64

Distribution of deliveries on time:
True     555652
False     40527
Name: delivery_on_time, dtype: int64


In [26]:
# create CSV from dataframe
ship_data.to_csv('Shipment.csv', encoding='utf-8-sig', index=True)